In [11]:
class GeneratorConfig:
    def __init__(self, entity_singular, entity_plural, prompt):
        self.entity_singular = entity_singular
        self.entity_plural = entity_plural
        self.prompt = prompt

In [2]:
from pathlib import Path

organization = "org-NaDNa5c5In5LspIF45qqArRr" # This is mine. You should change it!

keyfile = Path.home() / '.openai' / 'api.key'
with open(keyfile) as f:
    api_key = f.readline().strip()

import openai
openai.organization = organization
openai.api_key = api_key

gpt_model_name = "gpt-3.5-turbo"

def basic_completion(user_prompt, max_tokens=50):
    x = openai.ChatCompletion.create(
        model = gpt_model_name,
        messages = [
            {'role': 'user', 'content': user_prompt}
        ],
        max_tokens = max_tokens
    )
    return x.choices[0].message['content']

In [3]:
world_ped_prompt = """
Improvise a fictional fantasy world geography for a D&D 5e campaign setting.

The response should be formatted as markdown, in the following format.

# <World Name>

## <First Continent Name>

<First paragraph - Description of the continent's geography. Should desribe the major topographical features, regions, climates, and biomes. MUST NOT describe the inhabitants>

<Second paragraph - Description of the economic activity, degree of settlement, and races of the inhabitants for the regions in the continent.>

## <Second Continent Name>

<similar>

## <Third Continent Name>

<similar>
"""

world = basic_completion(world_ped_prompt, max_tokens=1024)
print(world)

# Arvandor

## Eldrida

Eldrida is a vast continent shaped like a crescent moon, cradling the Emerald Sea in its gentle arc. The north and east coasts are mountainous and rocky, dotted with glaciers and snow-capped peaks. The western and southern coasts are warmer and lush, with sandy beaches and dense forests. The central portion of the continent is a vast steppe that stretches from end to end, where the winds blow freely and nomadic tribes follow their herds.

The northern coast is the most prosperous region of Eldrida, with bustling port cities like Xanthar and Vernathia. These ancient cities are built on cliffs and overlook the sea, and have thrived on trade for centuries. The east coast is home to the dwarves of the Iron Mountains, who mine gold, silver, and precious gems from the rugged heights. To the west, the forests are inhabited by elves and the fey, who work together to create magical items and protect the natural balance. The southern coast is a wild frontier, where pirate

In [5]:
from mistletoe import Document

world_doc = Document(world)

raw_texts = [node.children[0].content for node in world_doc.children]

world_name = raw_texts[0]
world_name

i = 0
continent_raws = raw_texts[1:]
continents = []
while 3*i+2 < len(continent_raws):
    offset = 3*i
    name = continent_raws[offset]
    geography = continent_raws[offset + 1]
    economy = continent_raws[offset + 2]
    continents.append((name, geography, economy))
    i += 1

(world_name, continents)

('Arvandor',
 [('Eldrida',
   'Eldrida is a vast continent shaped like a crescent moon, cradling the Emerald Sea in its gentle arc. The north and east coasts are mountainous and rocky, dotted with glaciers and snow-capped peaks. The western and southern coasts are warmer and lush, with sandy beaches and dense forests. The central portion of the continent is a vast steppe that stretches from end to end, where the winds blow freely and nomadic tribes follow their herds.',
   'The northern coast is the most prosperous region of Eldrida, with bustling port cities like Xanthar and Vernathia. These ancient cities are built on cliffs and overlook the sea, and have thrived on trade for centuries. The east coast is home to the dwarves of the Iron Mountains, who mine gold, silver, and precious gems from the rugged heights. To the west, the forests are inhabited by elves and the fey, who work together to create magical items and protect the natural balance. The southern coast is a wild frontier, 

In [6]:
def continent_ped_prompt(continent): return f"""
Improvise the details of a continent's regions in a fantasy world for a D&D 5e campaign setting.

The continent's name is {(continent[0])}
{continent[1]}
{continent[2]}

The response should be formatted as markdown, in the following format.

# <Continent Name>

## <First Region Name>

<First paragraph - Description of the region's geography. Should describe geographical points of interest, climate, biome, and natural resources. MUST NOT describe the inhabitants.>

<Second paragraph - Description of the region's inhabitants. Should describe the races, major economic activities, and the major power factions.>

## <Second Region Name>

<similar>

...etc

"""

continent_response = basic_completion(continent_ped_prompt(continents[0]), max_tokens=1024)
print(continent_response)

# Eldrida

## Northern Coast

The Northern Coast of Eldrida is a rugged and mountainous region with heavy snowfall and bitter cold. The coast is dotted with glaciers and peppered with jagged peaks that descend steeply into the Emerald Sea. The region is known for its rich natural resources such as timber, furs, and precious stones, but it is the bustling port cities of Xanthar and Vernathia that drive the region's economy. These cities are built high on cliffs overlooking the sea, and have become wealthy by trading with the other nations of Eldrida.

The inhabitants of the Northern Coast are a mix of humans, dwarves, and gnomes who live in small towns throughout the region. The dwarves of the Iron Mountains to the east have carved out vast underground mines that yield gold, silver, and precious gems, which are traded in the coastal cities. The gnomes of the region use their expertise in mechanics and clockwork to create intricate devices that are often bought up by the wealthy merchant

In [7]:
from mistletoe import Document

continent_doc = Document(continent_response)

raw_texts = [node.children[0].content for node in continent_doc.children]

continent_name = raw_texts[0]
continent_name

i = 0
region_raws = raw_texts[1:]
regions = []
while 3*i+2 < len(region_raws):
    offset = 3*i
    name = region_raws[offset]
    geography = region_raws[offset + 1]
    economy = region_raws[offset + 2]
    regions.append((name, geography, economy))
    i += 1

(continent_name, regions)

('Eldrida',
 [('Northern Coast',
   "The Northern Coast of Eldrida is a rugged and mountainous region with heavy snowfall and bitter cold. The coast is dotted with glaciers and peppered with jagged peaks that descend steeply into the Emerald Sea. The region is known for its rich natural resources such as timber, furs, and precious stones, but it is the bustling port cities of Xanthar and Vernathia that drive the region's economy. These cities are built high on cliffs overlooking the sea, and have become wealthy by trading with the other nations of Eldrida.",
   "The inhabitants of the Northern Coast are a mix of humans, dwarves, and gnomes who live in small towns throughout the region. The dwarves of the Iron Mountains to the east have carved out vast underground mines that yield gold, silver, and precious gems, which are traded in the coastal cities. The gnomes of the region use their expertise in mechanics and clockwork to create intricate devices that are often bought up by the weal

In [8]:
def region_ped_prompt(region): return f"""
Improvise the details of a region's settlements in a fantasy world for a D&D 5e campaign setting.

The region's name is {region[0]}
{region[1]}
{region[2]}

The response should be formatted as markdown, in the following format.

# Region Name

## <First Settlement Name>

<First paragraph - Description of the settlement's location, economy, and size>

<Second paragraph - Description of the settlement's politics>

## <Second Settlement Name>

<similar>

...etc

"""

region_response = basic_completion(region_ped_prompt(regions[0]), max_tokens=1024)
print(region_response)

# Northern Coast

## Xanthar

Xanthar is a bustling port city built high on a cliff overlooking the Emerald Sea. Its main source of income is its thriving trade with other nations of Eldrida, thanks to its strategic location and wealthy merchants' guild. The city is populated mostly by humans and gnomes, who use their expertise in mechanics to create intricate clockwork devices that are in high demand. Xanthar is also known for its fine jewelry, which is made from the precious gems mined in the Iron Mountains to the east. 

Politically, Xanthar is ruled by the Xantharian Merchants' Guild, which has a firm grip on the city's resources and trade. The guild's leader is a shrewd and ambitious woman named Lirien, who is always on the lookout for new business opportunities and ways to outmaneuver her rivals. Despite her savvy business sense, Lirien is not well-liked by the common people of Xanthar, who see her as greedy and cutthroat.

## Vernathia

Vernathia is another port city located on 

In [9]:
# TODO - parameterize

oneshot_ped_prompt = """
Xanthar is a bustling port city built high on a cliff overlooking the Emerald Sea. Its main source of income is its thriving trade with other nations of Eldrida, thanks to its strategic location and wealthy merchants' guild. The city is populated mostly by humans and gnomes, who use their expertise in mechanics to create intricate clockwork devices that are in high demand. Xanthar is also known for its fine jewelry, which is made from the precious gems mined in the Iron Mountains to the east. 

Politically, Xanthar is ruled by the Xantharian Merchants' Guild, which has a firm grip on the city's resources and trade. The guild's leader is a shrewd and ambitious woman named Lirien, who is always on the lookout for new business opportunities and ways to outmaneuver her rivals. Despite her savvy business sense, Lirien is not well-liked by the common people of Xanthar, who see her as greedy and cutthroat.

Improvise the details of a one-shot adventure in Xanthar for a D&D 5e setting.

The difficulty of the adventure should be tailored to a group of level 1 characters.

The complexity and linearity of the adventure should be tailored to new players, and should not take more than 3 hours.

The response should have the following format:

# <Campaign Name>

## <Antagonist's Name>

<Paragraph - A description of the antagonists acting in the adventure, including specific hostile NPCs>

## <Name of Exposition Act>

<Paragraph - A scene where the players' characters meet eachother and an NPC delivers the quest>

<Paragraph - Some other notable friendly NPCs in the setting which provide hooks to guide the players to the first encounter>

## <Name of Rising Action Act>

<Paragraph - A simple encounter, with terrain and enemy details>

<Paragraph - A scene after the first encounter with an obvious hook to guide the players to the final encounter>

## <Name of Climax Act>

<Paragraph - A more challenging final encounter, with terrain and enemy details>
"""

oneshot_response = basic_completion(oneshot_ped_prompt, max_tokens=2048)
print(oneshot_response)

# The Heist of Xanthar

## Lirien, Leader of the Merchants' Guild

Lirien has recently acquired a precious jewel, the Heart of Xanthar, which she plans to sell to a wealthy client from across the Emerald Sea. However, a group of thieves have also set their sights on the jewel, and they plan to steal it before it is sold. The players will find themselves caught in the middle of this conflict, as they are hired by Lirien to guard the jewel during its transport.

## The Arrival

The players are at the docks of Xanthar, waiting for their contact, a gnome named Finni, who is supposed to give them the details of the job. However, Finni is nowhere to be seen. As the players start to worry, they are approached by Finnegan, a tavern owner and friend of Finni. Finnegan tells the players that Finni has been kidnapped by the thieves, and that they need to rescue him before it's too late.

## The Ambush

As the players make their way to the thieves' hideout, they are ambushed by a group of bandits.